In [131]:
import json

# Let's load Wilhelm's Translation of the I-Ching, and let's pour it into a JSON & CSV file

In `./iching_fulltext.md` we have a full english translation of the Wilhelm I-Ching translation. We want to write a simple parser which will take out all required pieces from the text and store it inside ann appropriately constructed JSON/CSV.

In [141]:
count = 0
with open('./iching_fulltext.md') as f:
    lines = f.readlines()


In [143]:
with open('./i-ching-basic.js') as json_file:
    data = json.load(json_file)


In [144]:
iching_full_text = ''.join(lines)

In [145]:
def iching_raw_text_section_split():
    start_ii = 10 # hex 1
    end_ii = 74 # hex 64

    iching_raw = {}
    iching_raw['hexagrams'] = {}
    split = iching_full_text.split("## ")

    for i in range(start_ii, end_ii):
        iching_raw['hexagrams'][i-start_ii] = [p.split("\n") for p in split][i]

    return iching_raw

#iching_raw['hexagrams'][0]

In [146]:
iching_raw = iching_raw_text_section_split()

In [147]:
keywords = [ 'above', 'below', 'THE JUDGMENT', 'THE IMAGE', 'THE LINES' ]

We want to do the following:
- identify lines with above keywords.
- If they are being identified, if all are identified, it's good news
- Then be sure to add binary notation, and PUIR-notation.

In [148]:
a = iching_raw['hexagrams'][1]

In [149]:
idx_array = {}
for i, l in enumerate(a):
    # print (i)
    for k in keywords:
        # print(k)
        if k in l:
            idx_array[k] = i

In [150]:
idx_array

{'above': 3, 'below': 5, 'THE JUDGMENT': 9, 'THE IMAGE': 22, 'THE LINES': 30}

In [151]:
print (get_basic_info(a, idx_array)+"\n")
print (retrieve_above(a, idx_array)+"\n")
print (retrieve_below(a, idx_array)+"\n")
print (retrieve_symbolic(a, idx_array)+"\n")
print (retrieve_judgment(a, idx_array)+"\n")
print (retrieve_image(a, idx_array)+"\n")
print (retrieve_lines(a, idx_array)+"\n")

2. K'un / The Receptive

above K'UN THE RECEPTIVE, EARTH

below K'UN THE RECEPTIVE, EARTH


This hexagram is made up of broken lines only. The broken lines represents the dark, yielding, receptive primal power of yin. The attribute of the hexagram is devotion; its image is the earth. It is the perfect complement of THE CREATIVE--the complement, not the opposite, for the Receptive does not combat the Creative but completes it . It represents nature in contrast to spirit, earth in contrast to heaven, space as against time, the female-maternal as against the male-paternal. However, as applied to human affairs, the principle of this complementary relationship is found not only in the relation between man and woman, but also in that between prince and minister and between father and son. Indeed, even in the individual this duality appears in the coexistence of the spiritual world and the world of the senses. But strictly speaking there is no real dualism here, because there is a clearly def

In [128]:
def get_basic_info(hex_text, indices):
    """  """
    return hex_text[0]

def retrieve_above(hex_text, indices):
    """  """
    above_text = hex_text[indices['above']]
    return above_text

def retrieve_below(hex_text, indices):
    """  """
    below_text = hex_text[indices['below']]
    return below_text

def retrieve_symbolic(hex_text, indices):
    """  """
    symbolic_text = '\n'.join( hex_text[indices['below']+1: indices['THE JUDGMENT']])
    return symbolic_text

def retrieve_judgment(hex_text, indices):
    """  """
    judgment_text = '\n'.join( hex_text[indices['THE JUDGMENT']+1: indices['THE IMAGE']])
    return judgment_text

def retrieve_image(hex_text, indices):
    """  """
    image_text = '\n'.join( hex_text[indices['THE IMAGE']+1: indices['THE LINES']])
    return image_text

def retrieve_lines(hex_text, indices):
    """ Retrieve symbolic language, and interpretation"""
    lines_text = '\n'.join( hex_text[indices['THE LINES']+1:])
    return lines_text

In [61]:
iching_raw['hexagrams'][1]

["2. K'un / The Receptive",
 '[Back to Index](#index)',
 '',
 "above K'UN THE RECEPTIVE, EARTH",
 '',
 "below K'UN THE RECEPTIVE, EARTH",
 '',
 'This hexagram is made up of broken lines only. The broken lines represents the dark, yielding, receptive primal power of yin. The attribute of the hexagram is devotion; its image is the earth. It is the perfect complement of THE CREATIVE--the complement, not the opposite, for the Receptive does not combat the Creative but completes it . It represents nature in contrast to spirit, earth in contrast to heaven, space as against time, the female-maternal as against the male-paternal. However, as applied to human affairs, the principle of this complementary relationship is found not only in the relation between man and woman, but also in that between prince and minister and between father and son. Indeed, even in the individual this duality appears in the coexistence of the spiritual world and the world of the senses. But strictly speaking there is

In [8]:
lines[:100]

['# Overview\n',
 '\n',
 '## Data Sources\n',
 '- Full I Ching Text by Richard Wilhelm from http://www2.unipr.it/~deyoung/I_Ching_Wilhelm_Translation.html\n',
 '\n',
 '- Foreword by Carl Gustav Jung from https://www.iging.com/intro/foreword.htm\n',
 '\n',
 '## About Richard Wilhelm 卫礼贤\n',
 '[https://schoolofwisdom.com/about/richard-wilhelm-one-of-the-school-of-wisdoms-most-notable-teachers/](https://schoolofwisdom.com/about/richard-wilhelm-one-of-the-school-of-wisdoms-most-notable-teachers/)\n',
 '\n',
 '![wilhelm](./img/wilhelm.jpg)\n',
 '\n',
 '## My Copy of the Book\n',
 '\n',
 '![My Copy of the Book](./img/my-copy.jpg)\n',
 '\n',
 '## Foreword by Carl Gustav Jung 荣格\n',
 '\n',
 '[Link to the Forward](./iching-foreword.html)\n',
 '\n',
 '## Trigrams\n',
 '\n',
 '![Trigrams](./img/trigrams.png)\n',
 '\n',
 'Source: wikipedia.com\n',
 '\n',
 '## Sequence Chart\n',
 '\n',
 '![Hexagram Sequence Chart](./img/sequence-chart.jpg)\n',
 '\n',
 'Source: inthefamilyway.org\n',
 '\n',
 '## Chi